# Endpoints

In [3]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
df_items = pd.read_parquet(r'C:\Users\fedez\OneDrive\Escritorio\PI-MLOps\Datasets\Procesado\items.parquet')
df_juegos = pd.read_parquet(r'C:\Users\fedez\OneDrive\Escritorio\PI-MLOps\Datasets\Procesado\juegos.parquet')
df_resenias = pd.read_parquet(r'C:\Users\fedez\OneDrive\Escritorio\PI-MLOps\Datasets\Procesado\resenias.parquet')

In [5]:
df_items.sample()

,user_id,items_count,item_id,playtime_forever
1109089,Tuffgroggz,107,498240,114


In [6]:
df_juegos.sample()

,genres,title,price,id,developer,release_year
1566,action,gotham city impostors free to play: weapon pac...,9.99,216432,"monolith productions, inc.",2012


In [7]:
df_resenias.sample()

,user_id,item_id,recommend,analisis_sentimiento
1635,76561198107811819,346120,True,2


In [8]:
# Se cambia el nombre de la columna "id" por "item_id" para poder realizar el merge de los dataframes
df_juegos.rename(columns={'id': 'item_id'}, inplace=True)
df_juegos.sample()

,genres,title,price,item_id,developer,release_year
20182,indie,armored kitten,2.99,713530,ducat,2017


### Se genran dataframes específicos para cada endpoint.

Endpoints 1, 4 y 5.  
Se genera un dataframe combinado para estos endpoints debido a que comparten varias columnas  
necesarias para cada función y el uso de memoria del archivo mantiene un tamaño óptimo.

In [9]:
# Se extraen las columnas necesarias de sus respectivos dataframes.
df_juegos_1_4_5 = df_juegos[['item_id', 'developer', 'release_year','price']]
df_resenias_1_4_5 = df_resenias[['item_id', 'analisis_sentimiento','recommend']]

# Se realiza el merge de las columnas extraídas.
df_fun_1_4_5 = df_resenias_1_4_5.merge(df_juegos_1_4_5, on='item_id', how='outer')

# Se procesa el dataframe para reducir el uso de memoria.
df_fun_1_4_5 = df_fun_1_4_5.drop_duplicates()
df_fun_1_4_5['price'] = df_fun_1_4_5['price'].fillna(0)
df_fun_1_4_5['price'] = df_fun_1_4_5['price'].astype(np.float16)
df_fun_1_4_5['release_year'] = df_fun_1_4_5['release_year'].fillna(0)
# Se eliminan las filas que tienen valores iguales a cero en la columna "playtime_forever" ya que no se trata de años.
df_fun_1_4_5 = df_fun_1_4_5[df_fun_1_4_5['release_year'] != 0]
df_fun_1_4_5['release_year'] = df_fun_1_4_5['release_year'].astype(np.int16)
df_fun_1_4_5['item_id'] = df_fun_1_4_5['item_id'].fillna(0)
# Se eliminan las filas con valores igual a cero em la columna "item_id" ya que no corresponden a un número id.
df_fun_1_4_5 = df_fun_1_4_5[df_fun_1_4_5['item_id'] != 0]
df_fun_1_4_5['item_id'] = df_fun_1_4_5['item_id'].astype(np.int32)
df_fun_1_4_5['analisis_sentimiento'] = df_fun_1_4_5['analisis_sentimiento'].fillna(0)
df_fun_1_4_5['analisis_sentimiento'] = df_fun_1_4_5['analisis_sentimiento'].astype(np.int32)

# Se hace un reset del índice.
df_fun_1_4_5.reset_index(inplace=True, drop=True)
df_fun_1_4_5.sample()

,item_id,analisis_sentimiento,recommend,developer,release_year,price
8889,371210,0,NaN,wareberd,2015,6.988281


In [10]:
df_fun_1_4_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25984 entries, 0 to 25983
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_id               25984 non-null  int32  
 1   analisis_sentimiento  25984 non-null  int32  
 2   recommend             6352 non-null   object 
 3   developer             25984 non-null  object 
 4   release_year          25984 non-null  int16  
 5   price                 25984 non-null  float16
dtypes: float16(1), int16(1), int32(2), object(2)
memory usage: 710.6+ KB


Endpoint 2.

In [11]:
# Se eliminan las columnas innecesarias de sus respectivos dataframes.
df_items_fun_2 = df_items.drop(['playtime_forever'], axis=1)
df_games_fun_2 = df_juegos.drop(['genres','title', 'developer','release_year'], axis=1)
df_reviews_fun_2 = df_resenias.drop(['analisis_sentimiento', 'item_id'], axis=1)

# Se combinan los dataframes resultantes.
df_merge = df_reviews_fun_2.merge(df_items_fun_2, on='user_id', how='outer')
df_fun_2 = df_merge.merge(df_games_fun_2, on='item_id', how='outer')

# Se procesa el dataframe para reducir el uso de memoria.
df_fun_2 = df_fun_2.drop_duplicates()
df_fun_2['items_count'] = df_fun_2['items_count'].fillna(0)
df_fun_2['items_count'] = df_fun_2['items_count'].astype(np.int32)
df_fun_2['price'] = df_fun_2['price'].fillna(0)
df_fun_2['price'] = df_fun_2['price'].astype(np.float16)
df_fun_2['item_id'] = df_fun_2['item_id'].fillna(0)
# Se eliminan las filas con valores igual a cero em la columna "item_id" ya que no corresponden a un número id.
df_fun_2 = df_fun_2[df_fun_2['item_id'] != 0]
df_fun_2['item_id'] = df_fun_2['item_id'].astype(np.int32)

#Se extrae una muestra del 10% del dataframe original para evitar complicaciones por uso de memoria en Render.
df_fun_2 = df_fun_2.sample(frac=0.1, random_state=42)

# Se hace un reset del índice.
df_fun_2.reset_index(inplace=True, drop=True)
df_fun_2.sample()

,user_id,recommend,items_count,item_id,price
468414,randymustache,True,231,34460,0.0


In [12]:
df_fun_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562441 entries, 0 to 562440
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      561011 non-null  object 
 1   recommend    309651 non-null  object 
 2   items_count  562441 non-null  int32  
 3   item_id      562441 non-null  int32  
 4   price        562441 non-null  float16
dtypes: float16(1), int32(2), object(2)
memory usage: 13.9+ MB


Endpoint 3.

In [13]:
# Se extraen las columnas necesarias de sus respectivos dataframes.
df_ju3 = df_juegos[['item_id', 'genres', 'release_year']]
df_it3 = df_items[['item_id', 'user_id','playtime_forever']]

# Se realiza el merge de las columnas extraídas.
df_fun_3 = df_ju3.merge(df_it3, on='item_id', how='outer')

# Se procesa el dataframe para reducir el uso de memoria.
df_fun_3['release_year'] = df_fun_3['release_year'].fillna(0)
# Se eliminan las filas que tienen valores iguales a cero en la columna "playtime_forever" ya que no se trata de años.
df_fun_3 = df_fun_3[df_fun_3['release_year'] != 0]
df_fun_3['release_year'] = df_fun_3['release_year'].astype(np.int16)
df_fun_3['playtime_forever'] = df_fun_3['playtime_forever'].fillna(0)
# Se eliminan las filas que tienen valores iguales a cero en la columna "playtime_forever" ya que no son necesarios para la función.
df_fun_3 = df_fun_3[df_fun_3['playtime_forever'] != 0]
df_fun_3['playtime_forever'] = df_fun_3['playtime_forever'].astype(np.int16)
df_fun_3['item_id'] = df_fun_3['item_id'].fillna(0)
# Se eliminan las filas con valores igual a cero em la columna "item_id" ya que no corresponden a un número id.
df_fun_3 = df_fun_3[df_fun_3['item_id'] != 0]
df_fun_3['item_id'] = df_fun_3['item_id'].astype(np.int32)

#Se extrae una muestra del 10% del dataframe original para evitar complicaciones por uso de memoria en Render.
df_fun_3 = df_fun_3.sample(frac=0.1, random_state=42)

# Se hace un reset del índice.
df_fun_3.reset_index(inplace=True, drop=True)
df_fun_3.sample()

,item_id,genres,release_year,user_id,playtime_forever
296921,227940,indie,2016,EnvyMouse,46


In [14]:
df_fun_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661138 entries, 0 to 661137
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   item_id           661138 non-null  int32 
 1   genres            661138 non-null  object
 2   release_year      661138 non-null  int16 
 3   user_id           661138 non-null  object
 4   playtime_forever  661138 non-null  int16 
dtypes: int16(2), int32(1), object(2)
memory usage: 15.1+ MB


### **Función 1**<br>
**def developer( desarrollador : str )**: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.<br>
Ejemplo de retorno:  
Año | Cantidad de Items | % de contenido Free  
2023 | 50 | 27%  
2022 | 45 | 25%  
xxxx | xx | xx%


In [19]:
def developer(desarrollador):
    
    # Se normaliza el valor convirtiéndolo a minúscula.
    desarrollador_m = desarrollador.lower()
    
    # Se fijan las filas en las que coincidan el desarrollador correspondiente.
    desarrolladora = df_fun_1_4_5[df_fun_1_4_5['developer'] == desarrollador_m]
    
    # Se genera una lista para guardar los resultados de las iteraciones.
    lista = []

    # Se itera sobre los años únicos en los que hay lanzamientos para la desarrolladora.
    for anio in desarrolladora['release_year'].unique():
        # Se filtran los ítems por año de lanzamiento.
        items_anio = desarrolladora[desarrolladora['release_year'] == anio]

        # Se filtran los ítems por año de lanzamiento.
        total_items = items_anio.shape[0]

        # Se cuentan el número de ítems con precio igual a 0.00 en ese año.
        items_gratis = items_anio[items_anio['price'] == 0.00].shape[0]

        # Se calcula el porcentaje de ítems gratuitos
        porcentaje_gratis = (items_gratis / total_items) * 100
        
        # Se agregan los resultados de las iteraciones a la lista
        lista.append({
            "Año": int(anio),
            "Cantidad de Items": int(total_items),
            "Contenido Free": f"{porcentaje_gratis:.2f}%"
        })

    # Se ordena la lista por el campo "Año" en orden descendente.
    lista_ordenada = sorted(lista, key=lambda x: x["Año"], reverse=True)

    return lista_ordenada

In [20]:
developer('Square Enix')

[{'Año': 2017, 'Cantidad de Items': 5, 'Contenido Free': '20.00%'},
 {'Año': 2016, 'Cantidad de Items': 4, 'Contenido Free': '0.00%'},
 {'Año': 2015, 'Cantidad de Items': 7, 'Contenido Free': '0.00%'},
 {'Año': 2014, 'Cantidad de Items': 16, 'Contenido Free': '0.00%'},
 {'Año': 2013, 'Cantidad de Items': 6, 'Contenido Free': '0.00%'},
 {'Año': 2010, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2009, 'Cantidad de Items': 4, 'Contenido Free': '0.00%'}]

In [49]:
developer('Valve')

[{'Año': 2012, 'Cantidad de Items': 6, 'Contenido Free': '0.00%'},
 {'Año': 2009, 'Cantidad de Items': 6, 'Contenido Free': '0.00%'},
 {'Año': 2004, 'Cantidad de Items': 22, 'Contenido Free': '0.00%'},
 {'Año': 2007, 'Cantidad de Items': 12, 'Contenido Free': '50.00%'},
 {'Año': 2011, 'Cantidad de Items': 6, 'Contenido Free': '0.00%'},
 {'Año': 1998, 'Cantidad de Items': 4, 'Contenido Free': '0.00%'},
 {'Año': 2008, 'Cantidad de Items': 4, 'Contenido Free': '0.00%'},
 {'Año': 2006, 'Cantidad de Items': 5, 'Contenido Free': '0.00%'},
 {'Año': 1999, 'Cantidad de Items': 5, 'Contenido Free': '0.00%'},
 {'Año': 2010, 'Cantidad de Items': 10, 'Contenido Free': '50.00%'},
 {'Año': 2000, 'Cantidad de Items': 6, 'Contenido Free': '0.00%'},
 {'Año': 2003, 'Cantidad de Items': 2, 'Contenido Free': '0.00%'},
 {'Año': 2001, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2017, 'Cantidad de Items': 2, 'Contenido Free': '0.00%'}]

### **Función 2**<br>
**def userdata( User_id : str )**: Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.<br>
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [15]:
def userdata(user_id):

    # Si el valor no es una cadena de texto se devuelve un mensaje de error.
    if type(user_id) != str:
        return 'Error: El valor ingresado debe ser una palabra'
    
    # Se normaliza el valor convirtiéndolo a minúscula.
    usuario_m = user_id.lower()
    
    # Se fijan las filas en las que coincidan el id de usuario correspondiente.
    usuario = df_fun_2[df_fun_2['user_id'] == usuario_m]

    # Se hace una sumatoria de los gatos respectivos a la variable.
    gasto = usuario['price'].sum()

    # Se obtiene el porcentaje de recomendación del usuario.
    if usuario['recommend'].sum() == 0:
        porcentaje_recomendacion = 0
    else:
        porcentaje_recomendacion =  (usuario['recommend'].sum() / len(usuario)) * 100

    # Se contabiliza la cantidad de items que tiene el usuario
    conteo_items = usuario.shape[0]

    datos_usuario = {
        "User X": user_id,
        "Dinero gastado": f"{gasto:.2f} USD",
        "Porcentaje de recomendación": f'{porcentaje_recomendacion}%',
        "Cantidad de items": conteo_items
        }
    
    return datos_usuario

In [16]:
userdata('us213ndjss09sdf')

{'User X': 'us213ndjss09sdf',
 'Dinero gastado': '0.00 USD',
 'Porcentaje de recomendación': '0%',
 'Cantidad de items': 0}

In [17]:
userdata('js41637')

{'User X': 'js41637',
 'Dinero gastado': '966.00 USD',
 'Porcentaje de recomendación': '100.0%',
 'Cantidad de items': 91}

### **Función 3**<br>
**def UserForGenre( genero : str )**: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.<br>
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [18]:
def User_For_Genre(genero):
    # Se convierte la cadena de texto ingresada en minúscula.
    genero_m = genero.lower()
    
    # Se corrobora que el valor ingresado esté dentro del dataframe.
    df_genero = df_fun_3[df_fun_3["genres"] == genero_m]

    # Se genera una sumatoria de las horas jugadas por año.
    df_horas_anuales = df_genero.groupby(["release_year"])["playtime_forever"].sum()
    df_horas_anuales = df_horas_anuales.reset_index()

    # Se hace una sumatoria de las horas respecto al usuario.
    df_horas = df_genero.groupby("user_id")["playtime_forever"].sum()
    
    # Se calcula el usuario con mayor cantidad de horas acumuladas.
    top_horas = df_horas.idxmax()

    df_horas_anuales = df_horas_anuales.rename(columns={"release_year": "Año", "playtime_forever": "Horas"})
    horas_anuales = df_horas_anuales.to_dict(orient="records")

    return {f"Usuario con más horas jugadas para género {genero}": top_horas, "Horas jugadas": horas_anuales}

In [19]:
User_For_Genre('action')

{'Usuario con más horas jugadas para género action': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 1983, 'Horas': 256},
  {'Año': 1988, 'Horas': 1162},
  {'Año': 1990, 'Horas': 2161},
  {'Año': 1991, 'Horas': 407},
  {'Año': 1992, 'Horas': 27},
  {'Año': 1993, 'Horas': 1874},
  {'Año': 1994, 'Horas': 17998},
  {'Año': 1995, 'Horas': 19660},
  {'Año': 1996, 'Horas': 4748},
  {'Año': 1997, 'Horas': 43243},
  {'Año': 1998, 'Horas': 246705},
  {'Año': 1999, 'Horas': 223067},
  {'Año': 2000, 'Horas': 676235},
  {'Año': 2001, 'Horas': 108493},
  {'Año': 2002, 'Horas': 58455},
  {'Año': 2003, 'Horas': 692589},
  {'Año': 2004, 'Horas': 5009527},
  {'Año': 2005, 'Horas': 1098669},
  {'Año': 2006, 'Horas': 963098},
  {'Año': 2007, 'Horas': 1602999},
  {'Año': 2008, 'Horas': 2392268},
  {'Año': 2009, 'Horas': 11179595},
  {'Año': 2010, 'Horas': 11300666},
  {'Año': 2011, 'Horas': 21282035},
  {'Año': 2012, 'Horas': 39341999},
  {'Año': 2013, 'Horas': 22170155},
  {'Año': 2014, 'Horas': 13685611},
  

### **Función 4**<br>
**def best_developer_year( año : int )**: Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos).<br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [20]:
def best_developer_year(anio):

    # Se corrobora que el input sea correcto.
    if anio not in df_fun_1_4_5['release_year'].unique():
        return f"El año {anio} no existe en los registros."
    
    # Filtrar el dataset para obtener solo las filas correspondientes al año dado.
    juegos_del_año = df_fun_1_4_5[df_fun_1_4_5['release_year'] == anio]

    # Se calculan las reseñas por desarrolladora.
    resenias = juegos_del_año.groupby('developer')['recommend'].sum().reset_index()

    # Se ordenan los juegos mejor valorados en orden ascendente.
    desarrolladoras = resenias.sort_values(by='recommend', ascending=False)

    # Se ordenan los primeros tres puestos
    oro = desarrolladoras.iloc[0]['developer']
    plata = desarrolladoras.iloc[1]['developer']
    bronce = desarrolladoras.iloc[2]['developer']

    # Crear la lista de diccionarios con los tres primeros lugares
    top3 = [{"Puesto 1": oro.title()}, {"Puesto 2": plata.title()}, {"Puesto 3": bronce.title()}]
    return top3

In [21]:
for i in range(2008, 2019):
    print(f'{i} : {best_developer_year(i)}')

2008 : [{'Puesto 1': 'Ubisoft Montreal'}, {'Puesto 2': 'Maxis™'}, {'Puesto 3': "Traveller'S Tales"}]
2009 : [{'Puesto 1': 'Popcap Games, Inc.'}, {'Puesto 2': 'Gearbox Software'}, {'Puesto 3': 'Ea Los Angeles'}]
2010 : [{'Puesto 1': 'Valve'}, {'Puesto 2': 'Bioware'}, {'Puesto 3': 'Obsidian Entertainment'}]
2011 : [{'Puesto 1': 'Obsidian Entertainment'}, {'Puesto 2': 'Reloaded Productions'}, {'Puesto 3': 'Volition'}]
2012 : [{'Puesto 1': 'The Behemoth'}, {'Puesto 2': 'Gearbox Software,Aspyr (Mac &Amp; Linux)'}, {'Puesto 3': 'Daedalic Entertainment'}]
2013 : [{'Puesto 1': 'Telltale Games'}, {'Puesto 2': 'Bohemia Interactive'}, {'Puesto 3': 'Deep Silver Volition'}]
2014 : [{'Puesto 1': 'Square Enix'}, {'Puesto 2': 'Ubisoft'}, {'Puesto 3': 'Telltale Games'}]
2015 : [{'Puesto 1': 'Psyonix, Inc.'}, {'Puesto 2': 'Capcom Co., Ltd.'}, {'Puesto 3': 'Koei Tecmo Games Co., Ltd.'}]
2016 : [{'Puesto 1': 'Bethesda Game Studios'}, {'Puesto 2': 'Capcom'}, {'Puesto 3': 'Paradox Development Studio'}]
2017

### **Función 5** <br>
**def developer_reviews_analysis( desarrolladora : str )**: Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.<br>
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [22]:
def developer_reviews_analysis(desarrolladora):

    # Si el valor no es una cadena de texto se devuelve un mensaje de error.
    if type(desarrolladora) != str:
        return 'Error: El valor ingresado debe ser una palabra'
    
    #Se convierten a minúscula los valores ingresados para compatibilizar con los valores de la columna 'developer'.
    desarrollador = desarrolladora.lower()

    #Se corrobora que el valor ingresado se encuentre en la columna.
    desarrollador = df_fun_1_4_5[df_fun_1_4_5['developer'] == desarrollador]
    
    analisis = desarrollador['analisis_sentimiento']
    # Se hace un conteo de las opiniones.
    opinion = analisis.value_counts()

    return {desarrolladora : list([f'Negative: {(opinion.get(0, 0))}',
                                    f'Positive: {(opinion.get(2, 0))}'])}

In [24]:
developer_reviews_analysis('Ubisoft')

{'Ubisoft': ['Negative: 23', 'Positive: 9']}

### Carga de datos.  
Se guardan los datasets específicos de los enpoints en archivos en formato parquet para su posterior uso.

Endpoints 1, 4 y 5.

In [25]:
table = pa.Table.from_pandas(df_fun_1_4_5)
pq.write_table(table, 'df_fun_1_4_5.parquet')

Endpoint 2

In [26]:
table = pa.Table.from_pandas(df_fun_2)
pq.write_table(table, 'df_fun_2.parquet')

Endpoint 3.

In [27]:
table = pa.Table.from_pandas(df_fun_3)
pq.write_table(table, 'df_fun_3.parquet')